# FTP Transfer of NYSE, NASDAQ and Other Market Tickers

In [16]:
from ftplib import FTP
import pandas_datareader.data as web
from datetime import datetime, timedelta
import pandas as pd
import pickle
import yfinance as yf

In [5]:
directory = 'symboldirectory'
filenames = ('otherlisted.txt', 'nasdaqlisted.txt')

ftp = FTP('ftp.nasdaqtrader.com')
ftp.login()
ftp.cwd(directory)

for item in filenames:
    ftp.retrbinary('RETR {0}'.format(item), open(item, 'wb').write)

ftp.quit()

# Create pandas dataframes from the nasdaqlisted and otherlisted files.
nasdaq_exchange_info = pd.read_csv('nasdaqlisted.txt', '|')
other_exchange_info = pd.read_csv('otherlisted.txt', '|')

C:\Users\Eddie\anaconda3\envs\securities-analysis\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
nasdaq_exchange_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4985 entries, 0 to 4984
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Symbol            4985 non-null   object 
 1   Security Name     4984 non-null   object 
 2   Market Category   4984 non-null   object 
 3   Test Issue        4984 non-null   object 
 4   Financial Status  4984 non-null   object 
 5   Round Lot Size    4984 non-null   float64
 6   ETF               4975 non-null   object 
 7   NextShares        4984 non-null   object 
dtypes: float64(1), object(7)
memory usage: 311.7+ KB


In [7]:
nasdaq_exchange_info

,Symbol,Security Name,Market Category,Test Issue,Financial Status,Round Lot Size,ETF,NextShares
0,AACG,ATA Creativity Global - American Depositary Sh...,G,N,N,100.0,N,N
1,AACIU,Armada Acquisition Corp. I - Unit,G,N,N,100.0,N,N
2,AADI,"Aadi Bioscience, Inc. - Common Stock",S,N,N,100.0,N,N
3,AADR,AdvisorShares Dorsey Wright ADR ETF,G,N,N,100.0,Y,N
4,AAL,"American Airlines Group, Inc. - Common Stock",Q,N,N,100.0,N,N
...,...,...,...,...,...,...,...,...
4980,ZXZZT,NASDAQ TEST STOCK,G,Y,N,100.0,NaN,N
4981,ZY,Zymergen Inc. - Common Stock,Q,N,N,100.0,N,N
4982,ZYNE,"Zynerba Pharmaceuticals, Inc. - Common Stock",G,N,N,100.0,N,N
4983,ZYXI,"Zynex, Inc. - Common Stock",Q,N,N,100.0,N,N


In [10]:
# contains the tickers from NYSE MKT (Exchange = A),
# New York Stock Exchange (NYSE) (Exchange = N),
# NYSE ARCA (Exchange = P),
# BATS Global Markets (Exchange = BATS),
# Investors' Exchange (Exchange = IEXG)
other_exchange_info

,ACT Symbol,Security Name,Exchange,CQS Symbol,ETF,Round Lot Size,Test Issue,NASDAQ Symbol
0,A,"Agilent Technologies, Inc. Common Stock",N,A,N,100.0,N,A
1,AA,Alcoa Corporation Common Stock,N,AA,N,100.0,N,AA
2,AAA,Listed Funds Trust AAF First Priority CLO Bond...,P,AAA,Y,100.0,N,AAA
3,AAAU,Goldman Sachs Physical Gold ETF Shares,P,AAAU,Y,100.0,N,AAAU
4,AAC,Ares Acquisition Corporation Class A Ordinary ...,N,AAC,N,100.0,N,AAC
...,...,...,...,...,...,...,...,...
6125,ZVIA,Zevia PBC Class A Common Stock,N,ZVIA,N,100.0,N,ZVIA
6126,ZVV,NYSE ARCA test stock,P,ZVV,N,100.0,Y,ZVV
6127,ZXIET,IEX Test Company Test Symbol Three for IEX,V,ZXIET,N,100.0,Y,ZXIET
6128,ZYME,Zymeworks Inc. Common Shares,N,ZYME,N,100.0,N,ZYME


### Find all stock symbols that are beating the S&P 500 by more 
### than 10% over the last 365 days and store the last 365 days worth
### of data for those stock tickers into a python pickle.
### Use SPY for the ETF which tracks the S&P 500

In [13]:
end = datetime.now()
start = end - timedelta(days=365)

valid_sources = ('morningstar', 'iex', 'yahoo')
close_string = 'Close'
source = valid_sources[2]

spy = web.DataReader('SPY', source, start, end)
spy_percent_change = (spy.iloc[-1][close_string] - spy.iloc[0][close_string]) / \
    spy.iloc[0][close_string] * 100

In [14]:
spy_percent_change

31.607874988830687

In [25]:
#nasdaq_exchange_info['Symbol']

nasdaq_tickers_str = ' '.join(list(nasdaq_exchange_info['Symbol']))

In [26]:
historical_data = yf.download(nasdaq_tickers_str)

[****                   8%                       ]  398 of 4988 completed

KeyboardInterrupt: 

[****                   8%                       ]  399 of 4988 completed

In [21]:
historical_data

Adj Close                   Close                    High  \
                   SPY        TSLA         SPY        TSLA         SPY   
Date                                                                     
1993-01-29   25.799770         NaN   43.937500         NaN   43.968750   
1993-02-01   25.983273         NaN   44.250000         NaN   44.250000   
1993-02-02   26.038315         NaN   44.343750         NaN   44.375000   
1993-02-03   26.313566         NaN   44.812500         NaN   44.843750   
1993-02-04   26.423655         NaN   45.000000         NaN   45.093750   
...                ...         ...         ...         ...         ...   
2021-09-03  453.079987  733.570007  453.079987  733.570007  453.630005   
2021-09-07  451.459991  752.919983  451.459991  752.919983  452.809998   
2021-09-08  450.910004  753.869995  450.910004  753.869995  451.670013   
2021-09-09  448.980011  754.859985  448.980011  754.859985  452.570007   
2021-09-10  445.440002  736.270020  445.440002  736.270020  451.489990   

                               Low                    Open              \
                  TSLA         SPY        TSLA         SPY        TSLA   
Date                                                                     
1993-01-29         NaN   43.750000         NaN   43.968750         NaN   
1993-02-01         NaN   43.968750         NaN   43.968750         NaN   
1993-02-02         NaN   44.125000         NaN   44.218750         NaN   
1993-02-03         NaN   44.375000         NaN   44.406250         NaN   
1993-02-04         NaN   44.468750         NaN   44.968750         NaN   
...                ...         ...         ...         ...         ...   
2021-09-03  734.000000  451.549988  724.200012  451.980011  732.250000   
2021-09-07  760.200012  450.739990  739.260010  452.709991  740.000000   
2021-09-08  764.450012  448.859985  740.770020  450.890015  761.580017   
2021-09-09  762.099976  448.720001  751.630005  450.700012  753.409973   
2021-09-10  762.609985  445.309998  734.520020  451.040009  759.599976   

              Volume              
                 SPY        TSLA  
Date                              
1993-01-29   1003200         NaN  
1993-02-01    480500         NaN  
1993-02-02    201300         NaN  
1993-02-03    529400         NaN  
1993-02-04    531500         NaN  
...              ...         ...  
2021-09-03  47170500  15246100.0  
2021-09-07  51671500  20039800.0  
2021-09-08  56181900  18793000.0  
2021-09-09  57970400  14077700.0  
2021-09-10  89848000  15114300.0  

[7207 rows x 12 columns]